# **Util functions**


The function *get_doc* is a function which takes as input a string representing a sentence and it returns as output a [*Doc*](https://spacy.io/api/doc).

A *Doc* is made up of a sequence of [*Token*](https://spacy.io/api/token).

Before to get the *Doc* instance, it is necessary to load a [*Language*](https://spacy.io/api/language/) through the method [*load*](https://spacy.io/usage/processing-pipelines).


In [1]:
import spacy
def get_doc(sentence):
  spacy_nlp = spacy.load('en_core_web_sm')
  return spacy_nlp(sentence)

# **Exercise 1** 

Exercise: *Extract a path of dependency relations from the ROOT to a token*

The function *path_of_dependencies* takes a string representing a sentence as input and return a list of list representing the path of dependency relations from the ROOT to a token for each token in the sentence.

The first necessary operation is to load the *Doc* instance by exploiting the function *get_doc* described in the previous section.

Then for each token in the sentence it extracts, it appends to the current list *tmp_list* the dependency relation of the current token by extracting its attribute [*dep_*](https://spacy.io/api/token#attributes), then it iterates from the token under analysis to the root by resorting to the attribute [*head*](https://spacy.io/usage/linguistic-features#navigating) of a token and for each token along the path it push the dependency relation of token crossed, after having reached the [*ROOT*](https://spacy.io/usage/linguistic-features#pos-tagging), it appends the entire current list to the list of list previously defined.

After having repeated this iterative cycle for all tokens, it returns the list of lists.

The reason behind the choice of a list of lists instead of a dictionary of lists is described in detail the "Report.pdf" file.



In [2]:
def path_of_dependencies(sentence):
  spacy_doc = get_doc(sentence)
  list_of_paths = []
  for token in spacy_doc:
    tmp_list = []
    tmp_list.append(token.dep_)
    while token.head != token:
      token = token.head
      tmp_list.insert(0,token.dep_)
    list_of_paths.append(tmp_list)
  return list_of_paths

# **Exercise 2**

Exercise: *Extract subtree of a dependents given a token*

The function *sorted_path_of_dependencies* takes as input a string representing a sentence and it returns as output a list of list.

The first operation is to load the *Doc* instance by using the function *get_doc* described in the "Util function" section.

Then for each token in the Doc, it exploit the attribute *subtree* of the current Token which returns a generator which is converted into a list by using the function *list* and this list is then appended to the list of all paths.

After having repeated this cycle over all tokens in the Doc, it returns the list of all paths.

The reason behind the choice of a list of list instead of a dictionary of list is described in detail the "Report.pdf" file.



In [3]:
def sorted_list_of_dependencies(sentence):
  spacy_doc = get_doc(sentence)
  sorted_list_of_paths = []
  for token in spacy_doc:
    sorted_list_of_paths.append(list(token.subtree))
  return sorted_list_of_paths


# **Exercise 3**

Exercise: *check if a given list of tokens (segment of a sentence) forms a subtree*

The function *seq_is_subtree* verifies if a sequence of tokens (described by the second argument *list_of_words*) represent a subtree of a string representing a sentence which is described by the first argument *sentence* and it returns as output a Boolean value.

As in the previous exercise, the first mandatory operation is to define the processed Doc by calling the *get_doc* function.

Then for each token in the Doc, it uses the attribute *subtree* of the current token, in order to return its subtree which is made up of generator of Token instances, then in order to convert this object into a string, I have defined the *gen_to_str* function which takes the subtree as input and convert each token into a string which is then appended into the list of strings which will be returned the *gen_to_str* function, then we verify if the list of strings returned by *gen_to_str* is equal to the list of tokens passed as input to the function *seq_is_subtree* by using the equality operator (*==*), if they are equal, the function returns *True* otherwise *False*

In [4]:
def gen_to_str(subtree):
    return [(str(it)) for it in subtree]


def seq_is_subtree(sentence, list_of_tokens):
  spacy_doc = get_doc(sentence)
  for token in spacy_doc:
    str_subtree = gen_to_str(token.subtree)
    if str_subtree == list_of_tokens:
      return True
  return False

# **Exercise 4**

Exercise: *identify head of a span, given its tokens*

The function *get_head* receives an input a string representing a *sentence* and  returns as output a  Token representing the head of the input sentece.

As in the  previous  exercises,  the  first operation  to  perform  is  the  Doc  processing by exploiting the *get_doc* function.

Then in  order to return the root, we need to store in a local variable the Span (list of tokens) contained in the Doc instance and finally return the [*root*](https://spacy.io/api/span#root) attribute of the [Span](https://spacy.io/api/span) instance representing the input sentence.


In [5]:
def get_head(sentence):
  doc = get_doc(sentence)
  span = doc[:]
  return span.root

# **Exercise 5**

Exercise: *Extract sentence subject, direct object and indirect object spans*

The function *extract_soi* receives as input a string representing the input sentence and returns a dictionary of lists as output.

The first thing to do is to define the processed Doc by using the *get_doc* function, then we need to declare a dictionary which, we will use to store the token of interest, made up of three different keys:
1. [*subj*](https://spacy.io/models/en#en_core_web_sm) representing the subjects
2. [*dobj*](https://spacy.io/models/en#en_core_web_sm) representing the direct objects
3. [*dative*](https://spacy.io/models/en#en_core_web_sm) representing the indirect objects

After this declaration, we have to iterate over all tokens in the input sentence
and check their attribute [*dep*](https://spacy.io/api/token#attributes) telling us the dependency relation.

We need to check if the current token is a subject, a direct object or an indirect object, if it is one of them, we append the current token to the list corresponding to that specific dependency relation defined by the *key*s previously described of the dictionary.

Finally, after having repeated this process for all tokens, we can return the dictionary of lists.

In [6]:
from collections import defaultdict

#soi represents "subject", "direct object" and "indirect object"
def extract_soi(sentence, look_for = ['nsubj','dobj','dative']):
  doc = get_doc(sentence)
  dict_of_spans = defaultdict(list)
  for it in look_for:
    dict_of_spans[it] = []
  for token in doc:
    print("token: {}, dep_rel: {}".format(token, token.dep_))
    for it in look_for:
      if token.dep_ == it:
        dict_of_spans[it].append(token)
  return dict_of_spans


#**Test**

*First exercise*

In [7]:
#these four sentences are the same of the file data.txt under the data directory 
list_of_sentences = ["I saw the man with a telescope.",
                     "The SMITH model produced by Google outperforms the BERT architecture.",
                     "The ELECTRA architecture uses Transformer and Generative Adversarial Networks.",
                     "Jim gave her the book."]

for sentence in list_of_sentences:
  print("\nThe sentence under analysis is : {}".format(sentence))
  list_of_paths = path_of_dependencies(sentence)
  doc = get_doc(sentence)
  for idx, it_path in enumerate(list_of_paths):
    print("token:{} - path:{}".format(doc[idx].text, it_path))


The sentence under analysis is : I saw the man with a telescope.
token:I - path:['ROOT', 'nsubj']
token:saw - path:['ROOT']
token:the - path:['ROOT', 'dobj', 'det']
token:man - path:['ROOT', 'dobj']
token:with - path:['ROOT', 'dobj', 'prep']
token:a - path:['ROOT', 'dobj', 'prep', 'pobj', 'det']
token:telescope - path:['ROOT', 'dobj', 'prep', 'pobj']
token:. - path:['ROOT', 'punct']

The sentence under analysis is : The SMITH model produced by Google outperforms the BERT architecture.
token:The - path:['ROOT', 'nsubj', 'det']
token:SMITH - path:['ROOT', 'nsubj', 'compound']
token:model - path:['ROOT', 'nsubj']
token:produced - path:['ROOT', 'nsubj', 'acl']
token:by - path:['ROOT', 'nsubj', 'acl', 'agent']
token:Google - path:['ROOT', 'compound']
token:outperforms - path:['ROOT']
token:the - path:['ROOT', 'dobj', 'det']
token:BERT - path:['ROOT', 'dobj', 'compound']
token:architecture - path:['ROOT', 'dobj']
token:. - path:['ROOT', 'punct']

The sentence under analysis is : The ELECTRA

*Second exercise*

In [8]:
for sentence in list_of_sentences:
  print("\nThe sentence under analysis is : {}".format(sentence))
  sorted_list_of_paths = sorted_list_of_dependencies(sentence)
  
  doc = get_doc(sentence)
  for (idx, it) in enumerate(sorted_list_of_paths):
    print("token:{} - sorted path:{}".format(doc[idx], it))


The sentence under analysis is : I saw the man with a telescope.
token:I - sorted path:[I]
token:saw - sorted path:[I, saw, the, man, with, a, telescope, .]
token:the - sorted path:[the]
token:man - sorted path:[the, man, with, a, telescope]
token:with - sorted path:[with, a, telescope]
token:a - sorted path:[a]
token:telescope - sorted path:[a, telescope]
token:. - sorted path:[.]

The sentence under analysis is : The SMITH model produced by Google outperforms the BERT architecture.
token:The - sorted path:[The]
token:SMITH - sorted path:[SMITH]
token:model - sorted path:[The, SMITH, model, produced, by]
token:produced - sorted path:[produced, by]
token:by - sorted path:[by]
token:Google - sorted path:[Google]
token:outperforms - sorted path:[The, SMITH, model, produced, by, Google, outperforms, the, BERT, architecture, .]
token:the - sorted path:[the]
token:BERT - sorted path:[BERT]
token:architecture - sorted path:[the, BERT, architecture]
token:. - sorted path:[.]

The sentence un

*Third exercise*

In [9]:
list_of_tokens = [[["saw","man","with"],["with","a","telescope"],["a"],["I","saw"]],
                  [["The","BERT","model"],["Google","outperforms"],["BERT",]],
                  [["The","ELECTRA","architecture"],["Generative","Adversarial","Networks"]],
                  [["Chomski"],["famous","linguists"],["Noam"]]]

for sentence, tokens in zip(list_of_sentences,list_of_tokens):
  for it in tokens:
    bool_answ = seq_is_subtree(sentence, it)
    if bool_answ == True:
      print("The list of tokens: {} is a subtree of the sentence \"{}\"".format(it, sentence))
    else:
      print("The list of tokens: {} is not a subtree of the sentence: \"{}\"".format(it, sentence))

The list of tokens: ['saw', 'man', 'with'] is not a subtree of the sentence: "I saw the man with a telescope."
The list of tokens: ['with', 'a', 'telescope'] is a subtree of the sentence "I saw the man with a telescope."
The list of tokens: ['a'] is a subtree of the sentence "I saw the man with a telescope."
The list of tokens: ['I', 'saw'] is not a subtree of the sentence: "I saw the man with a telescope."
The list of tokens: ['The', 'BERT', 'model'] is not a subtree of the sentence: "The SMITH model produced by Google outperforms the BERT architecture."
The list of tokens: ['Google', 'outperforms'] is not a subtree of the sentence: "The SMITH model produced by Google outperforms the BERT architecture."
The list of tokens: ['BERT'] is a subtree of the sentence "The SMITH model produced by Google outperforms the BERT architecture."
The list of tokens: ['The', 'ELECTRA', 'architecture'] is a subtree of the sentence "The ELECTRA architecture uses Transformer and Generative Adversarial Ne

*Forth exercise*

In [10]:
for sentence in list_of_sentences:
  print("The root of: \"{}\" is: '{}'".format(sentence, get_head(sentence)))

The root of: "I saw the man with a telescope." is: 'saw'
The root of: "The SMITH model produced by Google outperforms the BERT architecture." is: 'outperforms'
The root of: "The ELECTRA architecture uses Transformer and Generative Adversarial Networks." is: 'uses'
The root of: "Jim gave her the book." is: 'gave'


Fifth exercise

In [11]:
for sentence in list_of_sentences:
  dict_soi = extract_soi(sentence)
  print("Sentence under analysis: \"{}\"".format(sentence))
  for it in dict_soi:
    print("{}: {}".format(it, dict_soi[it]))



sentence2="Joe gave Jim the ball"
dict_soi = extract_soi(sentence2)

print("The second sentence is: {}".format(sentence2))
for it in dict_soi:
  print("{}: {}".format(it, dict_soi[it]))


token: I, dep_rel: nsubj
token: saw, dep_rel: ROOT
token: the, dep_rel: det
token: man, dep_rel: dobj
token: with, dep_rel: prep
token: a, dep_rel: det
token: telescope, dep_rel: pobj
token: ., dep_rel: punct
Sentence under analysis: "I saw the man with a telescope."
nsubj: [I]
dobj: [man]
dative: []
token: The, dep_rel: det
token: SMITH, dep_rel: compound
token: model, dep_rel: nsubj
token: produced, dep_rel: acl
token: by, dep_rel: agent
token: Google, dep_rel: compound
token: outperforms, dep_rel: ROOT
token: the, dep_rel: det
token: BERT, dep_rel: compound
token: architecture, dep_rel: dobj
token: ., dep_rel: punct
Sentence under analysis: "The SMITH model produced by Google outperforms the BERT architecture."
nsubj: [model]
dobj: [architecture]
dative: []
token: The, dep_rel: det
token: ELECTRA, dep_rel: compound
token: architecture, dep_rel: nsubj
token: uses, dep_rel: ROOT
token: Transformer, dep_rel: dobj
token: and, dep_rel: cc
token: Generative, dep_rel: compound
token: Adver

# **Advanced and Optional part**

Exercise 1: *Modify NLTK Transition parser's Configuration class to use better features*

In order to improve the model performance, I have added the following features to the result string in the function *extract_features*:
 the Levenshtein distance between the word on the topof the stack and the 

1.   Levenshtein distance between the word on the top of the stack and the last word in the buffer

2.   The left child of the first node in the buffer

3.   The right child of the first node in the buffer

4.  The index of the element in the buffer taken in consideration

5.  The lenght of the buffer

These features added have been inspired by the [PAPER], the Stanford's Natural Language Processing course and some empirical tries.

In order to add these features to the original ones, it is required to specify the third argument of the *train* function (*modified_features=True*) and the third argument of the *parse* function (*modified_features=True*).

N.B. Modified features is set to *False* as default value.

Exercise 2: *Evaluate the features comparing performance to the original*

In order to evaluate the features, it is required to train and test the model by specifying the third argument of the train function and the third argument of parse function (*modified_features*) as *True* or *False* 

Exercise 3:*Replace SVM classifier with an alternative of your choice*

In the *Train* function, I have added an additional parameter (*model_spec*) in order to specify the model to be used during training.

The default value is (*spec_mod='svm'*) in order to use the SVM model.

The other model supported (and which has been tested) are:

1.   Decision Tree Classifier (*spec_mod = 'DecisionTreeClassifier'*)
2.   Random Forest (*spec_mod = 'RandomForestClassifier'*)
3.   Multi-Layer Perceptron (*spec_mod = 'MLPClassifier'*)

The model which provides the best performance has been the *Decision Tree Classifier* which guarantees results pretty similar to the SVM but in much less training time.

For detailed analysis about the performance of the different models (by using the original features or the modified features) refer to the file 'Report.pdf'.

NB: In order to simplify the reading of the added part with respect to the original Configuration class, I have added as comments: "#BEGIN ADDED CODE pt." at the beginning of the additional lines of code and "#END ADDED CODE pt." at the end, while about the modifications in the argument passed to the function I have added as comment:"#THE ARGUMENTS ARE CHANGED"






In [12]:
# Natural Language Toolkit: Arc-Standard and Arc-eager Transition Based Parsers
import tempfile
import pickle

from os import remove
from copy import deepcopy
from operator import itemgetter

try:
    from numpy import array
    from scipy import sparse
    from sklearn.datasets import load_svmlight_file
    from sklearn import svm
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
except ImportError:
    pass

from nltk.parse import ParserI, DependencyGraph, DependencyEvaluator


class Configuration(object):
    """
    Class for holding configuration which is the partial analysis of the input sentence.
    The transition based parser aims at finding set of operators that transfer the initial
    configuration to the terminal configuration.
    The configuration includes:
        - Stack: for storing partially proceeded words
        - Buffer: for storing remaining input words
        - Set of arcs: for storing partially built dependency tree
    This class also provides a method to represent a configuration as list of features.
    """

    def __init__(self, dep_graph):
        """
        :param dep_graph: the representation of an input in the form of dependency graph.
        :type dep_graph: DependencyGraph where the dependencies are not specified.
        """
        # dep_graph.nodes contain list of token for a sentence
        self.stack = [0]  # The root element
        self.buffer = list(range(1, len(dep_graph.nodes)))  # The rest is in the buffer
        self.arcs = []  # empty set of arc
        self._tokens = dep_graph.nodes
        self._max_address = len(self.buffer)
        #BEGIN ADDED CODE pt.0
        self.dg = dep_graph
        #END ADDED CODE pt.0      

    def __str__(self):
        return (
            "Stack : "
            + str(self.stack)
            + "  Buffer : "
            + str(self.buffer)
            + "   Arcs : "
            + str(self.arcs)
        )

    def _check_informative(self, feat, flag=False):
        """
        Check whether a feature is informative
        The flag control whether "_" is informative or not
        """
        if feat is None:
            return False
        if feat == "":
            return False
        if flag is False:
            if feat == "_":
                return False
        return True

    #THE ARGUMENTS ARE CHANGED
    def extract_features(self, modified_features=False):
        result = []

        #BEGIN ADDED CODE pt.1
        stack_top_token = None
        stack_idx = None
        #END ADDED CODE pt.1

        if len(self.stack) > 0:
            # Stack 0
            stack_idx0 = self.stack[len(self.stack) - 1]
            token = self._tokens[stack_idx0]
            #BEGIN ADDED CODE pt.2
            stack_top_token = token["word"]   
            stack_idx = stack_idx0
            #END ADDED CODE pt.2
            if self._check_informative(token["word"], True):
                result.append("STK_0_FORM_" + token["word"])
            if "lemma" in token and self._check_informative(token["lemma"]):
                result.append("STK_0_LEMMA_" + token["lemma"])
            if self._check_informative(token["tag"]):
                result.append("STK_0_POS_" + token["tag"])
            if "feats" in token and self._check_informative(token["feats"]):
                feats = token["feats"].split("|")
                for feat in feats:
                    result.append("STK_0_FEATS_" + feat)
            # Stack 1
            if len(self.stack) > 1:
                stack_idx1 = self.stack[len(self.stack) - 2]
                token = self._tokens[stack_idx1]
                if self._check_informative(token["tag"]):
                    result.append("STK_1_POS_" + token["tag"])

            # Left most, right most dependency of stack[0]
            left_most = 1000000
            right_most = -1
            dep_left_most = ""
            dep_right_most = ""
            for (wi, r, wj) in self.arcs:
                if wi == stack_idx0:
                    if (wj > wi) and (wj > right_most):
                        right_most = wj
                        dep_right_most = r
                    if (wj < wi) and (wj < left_most):
                        left_most = wj
                        dep_left_most = r
            if self._check_informative(dep_left_most):
                result.append("STK_0_LDEP_" + dep_left_most)
            if self._check_informative(dep_right_most):
                result.append("STK_0_RDEP_" + dep_right_most)

        # Check Buffered 0

        #BEGIN ADDED CODE pt.3
        buffer_first_token = None
        buffer_idx = None
        #END ADDED CODE pt.3

        if len(self.buffer) > 0:
            # Buffer 0
            buffer_idx0 = self.buffer[0]
            token = self._tokens[buffer_idx0]
            #BEGIN ADDED CODE pt.4
            buffer_idx = buffer_idx0
            buffer_first_token = token["word"]
            #END ADDED CODE pt.4
            if self._check_informative(token["word"], True):
                result.append("BUF_0_FORM_" + token["word"])
            if "lemma" in token and self._check_informative(token["lemma"]):
                result.append("BUF_0_LEMMA_" + token["lemma"])
            if self._check_informative(token["tag"]):
                result.append("BUF_0_POS_" + token["tag"])
            if "feats" in token and self._check_informative(token["feats"]):
                feats = token["feats"].split("|")
                for feat in feats:
                    result.append("BUF_0_FEATS_" + feat)
            # Buffer 1
            if len(self.buffer) > 1:
                buffer_idx1 = self.buffer[1]
                token = self._tokens[buffer_idx1]            
                if self._check_informative(token["word"], True):
                    result.append("BUF_1_FORM_" + token["word"])
                if self._check_informative(token["tag"]):
                    result.append("BUF_1_POS_" + token["tag"])
            if len(self.buffer) > 2:
                buffer_idx2 = self.buffer[2]
                token = self._tokens[buffer_idx2]    
                if self._check_informative(token["tag"]):
                    result.append("BUF_2_POS_" + token["tag"])
            if len(self.buffer) > 3:
                buffer_idx3 = self.buffer[3]
                token = self._tokens[buffer_idx3]   
                if self._check_informative(token["tag"]):
                    result.append("BUF_3_POS_" + token["tag"])
                    # Left most, right most dependency of stack[0]
            left_most = 1000000
            right_most = -1
            dep_left_most = ""
            dep_right_most = ""
            for (wi, r, wj) in self.arcs:
                if wi == buffer_idx0:
                    if (wj > wi) and (wj > right_most):
                        right_most = wj
                        dep_right_most = r
                    if (wj < wi) and (wj < left_most):
                        left_most = wj
                        dep_left_most = r
            if self._check_informative(dep_left_most):
                result.append("BUF_0_LDEP_" + dep_left_most)
            if self._check_informative(dep_right_most):
                result.append("BUF_0_RDEP_" + dep_right_most)
        #BEGIN ADDED CODE pt.5  
        if modified_features == True:
          #Compute distance
          from pytextdist.edit_distance import levenshtein_distance
          if buffer_first_token is not None and stack_top_token is not None:
            result.append("DIS_"+str(levenshtein_distance(buffer_first_token, stack_top_token)))
          else:
            result.append("DIS_"+str(0))
          if buffer_idx is not None:
            result.append("BI_"+str(buffer_idx))
            result.append("BLC_"+str(self.dg.left_children(buffer_idx)))
            result.append("BRC_"+str(self.dg.right_children(buffer_idx)))
          else:
            result.append("BI_"+str(0))
            result.append("BLC_"+str(0))
            result.append("BRC_"+str(0))
          result.append("BUF_L_"+str(len(self.buffer)))

          if "deps" in token and len(token["deps"])>0:
               for dep in token["deps"]:
                 result.append("DEPS_"+str(dep))
          #END ADDED CODE pt.5
          
          #if stack_idx is not None:
          #  result.append("SI"+str(stack_idx))
          #  result.append("SLC"+str(self.dg.left_children(stack_idx)))
          #  result.append("SRC"+str(self.dg.right_children(stack_idx)))
          #else:
          #  result.append("SI"+str(0))
          #  result.append("SLC"+str(0))
          #  result.append("SRC"+str(0))
          #result.append("STACK LEN"+str(len(self.stack)))
        return result


class Transition(object):
    """
    This class defines a set of transition which is applied to a configuration to get another configuration
    Note that for different parsing algorithm, the transition is different.
    """

    # Define set of transitions
    LEFT_ARC = "LEFTARC"
    RIGHT_ARC = "RIGHTARC"
    SHIFT = "SHIFT"
    REDUCE = "REDUCE"

    def __init__(self, alg_option):
        """
        :param alg_option: the algorithm option of this parser. Currently support `arc-standard` and `arc-eager` algorithm
        :type alg_option: str
        """
        self._algo = alg_option
        if alg_option not in [
            TransitionParser.ARC_STANDARD,
            TransitionParser.ARC_EAGER,
        ]:
            raise ValueError(
                " Currently we only support %s and %s "
                % (TransitionParser.ARC_STANDARD, TransitionParser.ARC_EAGER)
            )

    def left_arc(self, conf, relation):
        """
        Note that the algorithm for left-arc is quite similar except for precondition for both arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if (len(conf.buffer) <= 0) or (len(conf.stack) <= 0):
            return -1
        if conf.buffer[0] == 0:
            # here is the Root element
            return -1

        idx_wi = conf.stack[len(conf.stack) - 1]

        flag = True
        if self._algo == TransitionParser.ARC_EAGER:
            for (idx_parent, r, idx_child) in conf.arcs:
                if idx_child == idx_wi:
                    flag = False

        if flag:
            conf.stack.pop()
            idx_wj = conf.buffer[0]
            conf.arcs.append((idx_wj, relation, idx_wi))
        else:
            return -1

    def right_arc(self, conf, relation):
        """
        Note that the algorithm for right-arc is DIFFERENT for arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if (len(conf.buffer) <= 0) or (len(conf.stack) <= 0):
            return -1
        if self._algo == TransitionParser.ARC_STANDARD:
            idx_wi = conf.stack.pop()
            idx_wj = conf.buffer[0]
            conf.buffer[0] = idx_wi
            conf.arcs.append((idx_wi, relation, idx_wj))
        else:  # arc-eager
            idx_wi = conf.stack[len(conf.stack) - 1]
            idx_wj = conf.buffer.pop(0)
            conf.stack.append(idx_wj)
            conf.arcs.append((idx_wi, relation, idx_wj))

    def reduce(self, conf):
        """
        Note that the algorithm for reduce is only available for arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """

        if self._algo != TransitionParser.ARC_EAGER:
            return -1
        if len(conf.stack) <= 0:
            return -1

        idx_wi = conf.stack[len(conf.stack) - 1]
        flag = False
        for (idx_parent, r, idx_child) in conf.arcs:
            if idx_child == idx_wi:
                flag = True
        if flag:
            conf.stack.pop()  # reduce it
        else:
            return -1

    def shift(self, conf):
        """
        Note that the algorithm for shift is the SAME for arc-standard and arc-eager
            :param configuration: is the current configuration
            :return : A new configuration or -1 if the pre-condition is not satisfied
        """
        if len(conf.buffer) <= 0:
            return -1
        idx_wi = conf.buffer.pop(0)
        conf.stack.append(idx_wi)


class TransitionParser(ParserI):

    """
    Class for transition based parser. Implement 2 algorithms which are "arc-standard" and "arc-eager"
    """

    ARC_STANDARD = "arc-standard"
    ARC_EAGER = "arc-eager"

    def __init__(self, algorithm):
        """
        :param algorithm: the algorithm option of this parser. Currently support `arc-standard` and `arc-eager` algorithm
        :type algorithm: str
        """
        if not (algorithm in [self.ARC_STANDARD, self.ARC_EAGER]):
            raise ValueError(
                " Currently we only support %s and %s "
                % (self.ARC_STANDARD, self.ARC_EAGER)
            )
        self._algorithm = algorithm

        self._dictionary = {}
        self._transition = {}
        self._match_transition = {}

    def _get_dep_relation(self, idx_parent, idx_child, depgraph):
        p_node = depgraph.nodes[idx_parent]
        c_node = depgraph.nodes[idx_child]

        if c_node["word"] is None:
            return None  # Root word

        if c_node["head"] == p_node["address"]:
            return c_node["rel"]
        else:
            return None

    def _convert_to_binary_features(self, features):
        """
        :param features: list of feature string which is needed to convert to binary features
        :type features: list(str)
        :return : string of binary features in libsvm format  which is 'featureID:value' pairs
        """
        unsorted_result = []
        for feature in features:
            self._dictionary.setdefault(feature, len(self._dictionary))
            unsorted_result.append(self._dictionary[feature])

        # Default value of each feature is 1.0
        return " ".join(
            str(featureID) + ":1.0" for featureID in sorted(unsorted_result)
        )

    def _is_projective(self, depgraph):
        arc_list = []
        for key in depgraph.nodes:
            node = depgraph.nodes[key]

            if "head" in node:
                childIdx = node["address"]
                parentIdx = node["head"]
                if parentIdx is not None:
                    arc_list.append((parentIdx, childIdx))

        for (parentIdx, childIdx) in arc_list:
            # Ensure that childIdx < parentIdx
            if childIdx > parentIdx:
                temp = childIdx
                childIdx = parentIdx
                parentIdx = temp
            for k in range(childIdx + 1, parentIdx):
                for m in range(len(depgraph.nodes)):
                    if (m < childIdx) or (m > parentIdx):
                        if (k, m) in arc_list:
                            return False
                        if (m, k) in arc_list:
                            return False
        return True

    def _write_to_file(self, key, binary_features, input_file):
        """
        write the binary features to input file and update the transition dictionary
        """
        self._transition.setdefault(key, len(self._transition) + 1)
        self._match_transition[self._transition[key]] = key

        input_str = str(self._transition[key]) + " " + binary_features + "\n"
        input_file.write(input_str.encode("utf-8"))

    #THE ARGUMENTS ARE CHANGED
    def _create_training_examples_arc_std(self, depgraphs, input_file, modified_features=False):
        """
        Create the training example in the libsvm format and write it to the input_file.
        Reference : Page 32, Chapter 3. Dependency Parsing by Sandra Kubler, Ryan McDonal and Joakim Nivre (2009)
        """
        operation = Transition(self.ARC_STANDARD)
        count_proj = 0
        training_seq = []

        for depgraph in depgraphs:
            if not self._is_projective(depgraph):
                continue

            count_proj += 1
            conf = Configuration(depgraph)
            while len(conf.buffer) > 0:
                b0 = conf.buffer[0]
                features = conf.extract_features(modified_features)
                binary_features = self._convert_to_binary_features(features)

                if len(conf.stack) > 0:
                    s0 = conf.stack[len(conf.stack) - 1]
                    # Left-arc operation
                    rel = self._get_dep_relation(b0, s0, depgraph)
                    if rel is not None:
                        key = Transition.LEFT_ARC + ":" + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.left_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # Right-arc operation
                    rel = self._get_dep_relation(s0, b0, depgraph)
                    if rel is not None:
                        precondition = True
                        # Get the max-index of buffer
                        maxID = conf._max_address

                        for w in range(maxID + 1):
                            if w != b0:
                                relw = self._get_dep_relation(b0, w, depgraph)
                                if relw is not None:
                                    if (b0, relw, w) not in conf.arcs:
                                        precondition = False

                        if precondition:
                            key = Transition.RIGHT_ARC + ":" + rel
                            self._write_to_file(key, binary_features, input_file)
                            operation.right_arc(conf, rel)
                            training_seq.append(key)
                            continue

                # Shift operation as the default
                key = Transition.SHIFT
                self._write_to_file(key, binary_features, input_file)
                operation.shift(conf)
                training_seq.append(key)

        return training_seq


    #THE ARGUMENTS ARE CHANGED
    def _create_training_examples_arc_eager(self, depgraphs, input_file, modified_features = False):
        """
        Create the training example in the libsvm format and write it to the input_file.
        Reference : 'A Dynamic Oracle for Arc-Eager Dependency Parsing' by Joav Goldberg and Joakim Nivre
        """
        operation = Transition(self.ARC_EAGER)
        countProj = 0
        training_seq = []

        for depgraph in depgraphs:
            if not self._is_projective(depgraph):
                continue

            countProj += 1
            conf = Configuration(depgraph)
            while len(conf.buffer) > 0:
                b0 = conf.buffer[0]
                features = conf.extract_features(modified_features)
                binary_features = self._convert_to_binary_features(features)

                if len(conf.stack) > 0:
                    s0 = conf.stack[len(conf.stack) - 1]
                    # Left-arc operation
                    rel = self._get_dep_relation(b0, s0, depgraph)
                    if rel is not None:
                        key = Transition.LEFT_ARC + ":" + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.left_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # Right-arc operation
                    rel = self._get_dep_relation(s0, b0, depgraph)
                    if rel is not None:
                        key = Transition.RIGHT_ARC + ":" + rel
                        self._write_to_file(key, binary_features, input_file)
                        operation.right_arc(conf, rel)
                        training_seq.append(key)
                        continue

                    # reduce operation
                    flag = False
                    for k in range(s0):
                        if self._get_dep_relation(k, b0, depgraph) is not None:
                            flag = True
                        if self._get_dep_relation(b0, k, depgraph) is not None:
                            flag = True
                    if flag:
                        key = Transition.REDUCE
                        self._write_to_file(key, binary_features, input_file)
                        operation.reduce(conf)
                        training_seq.append(key)
                        continue

                # Shift operation as the default
                key = Transition.SHIFT
                self._write_to_file(key, binary_features, input_file)
                operation.shift(conf)
                training_seq.append(key)

        return training_seq

    #Additional parameter: "spec_model" describing the model of interest
    def train(self, depgraphs, modelfile, verbose=True, modified_features = False, spec_mod="svm"):
        """
        :param depgraphs : list of DependencyGraph as the training data
        :type depgraphs : DependencyGraph
        :param modelfile : file name to save the trained model
        :type modelfile : str
        """

        try:
            input_file = tempfile.NamedTemporaryFile(
                prefix="transition_parse.train", dir=tempfile.gettempdir(), delete=False
            )

            if self._algorithm == self.ARC_STANDARD:
                self._create_training_examples_arc_std(depgraphs, input_file, modified_features)
            else:
                self._create_training_examples_arc_eager(depgraphs, input_file, modified_features)

            input_file.close()
            
            # Using the temporary file to train the libsvm classifier
            x_train, y_train = load_svmlight_file(input_file.name)


            #BEGIN ADDED CODE PT.6
            if (spec_mod == 'DecisionTreeClassifier'):
              model = DecisionTreeClassifier()
              #the default criterion ('gini') [accuracy=0.89]
              #works better than the ('entropy') criterion [accuracy=0.87] 
            
            elif (spec_mod == 'KNeighborsClassifier'):
              from sklearn.neighbors import KNeighborsClassifier
              model = KNeighborsClassifier(n_neighbors=3)

            elif (spec_mod == 'RandomForestClassifier'):
              model = RandomForestClassifier(random_state=0)

            elif (spec_mod == 'MLPClassifier'):
              from sklearn.neural_network import MLPClassifier
              model = MLPClassifier(random_state=4, max_iter=300)

            else:
              print('svm')
              # The parameter is set according to the paper:
              # Algorithms for Deterministic Incremental Dependency Parsing by Joakim Nivre
              # Todo : because of probability = True => very slow due to
              # cross-validation. Need to improve the speed here
              model = svm.SVC(
                kernel="poly",
                degree=2,
                coef0=0,
                gamma=0.2,
                C=0.5,
                verbose=verbose,
                probability=True,
                )
              
            model = model.fit(x_train, y_train)
            #END ADDED CODE PT.6


            # Save the model to file name (as pickle)
            pickle.dump(model, open(modelfile, "wb"))
        
        
        finally:
            remove(input_file.name)


    def parse(self, depgraphs, modelFile, modified_features=False):
        """
        :param depgraphs: the list of test sentence, each sentence is represented as a dependency graph where the 'head' information is dummy
        :type depgraphs: list(DependencyGraph)
        :param modelfile: the model file
        :type modelfile: str
        :return: list (DependencyGraph) with the 'head' and 'rel' information
        """
        result = []
        # First load the model
        model = pickle.load(open(modelFile, "rb"))
        operation = Transition(self._algorithm)

        for depgraph in depgraphs:
            conf = Configuration(depgraph)
            while len(conf.buffer) > 0:
                features = conf.extract_features(modified_features)
                col = []
                row = []
                data = []
                for feature in features:
                    if feature in self._dictionary:
                        col.append(self._dictionary[feature])
                        row.append(0)
                        data.append(1.0)
                np_col = array(sorted(col))  # NB : index must be sorted
                np_row = array(row)
                np_data = array(data)

                x_test = sparse.csr_matrix(
                    (np_data, (np_row, np_col)), shape=(1, len(self._dictionary))
                )


                prob_dict = {}
                pred_prob = model.predict_proba(x_test)[0]
                for i in range(len(pred_prob)):
                    prob_dict[i] = pred_prob[i]
                sorted_Prob = sorted(prob_dict.items(), key=itemgetter(1), reverse=True)

                # Note that SHIFT is always a valid operation
                for (y_pred_idx, confidence) in sorted_Prob:
                    # y_pred = model.predict(x_test)[0]
                    # From the prediction match to the operation
                    y_pred = model.classes_[y_pred_idx]

                    if y_pred in self._match_transition:
                        strTransition = self._match_transition[y_pred]
                        baseTransition = strTransition.split(":")[0]

                        if baseTransition == Transition.LEFT_ARC:
                            if (
                                operation.left_arc(conf, strTransition.split(":")[1])
                                != -1
                            ):
                                break
                        elif baseTransition == Transition.RIGHT_ARC:
                            if (
                                operation.right_arc(conf, strTransition.split(":")[1])
                                != -1
                            ):
                                break
                        elif baseTransition == Transition.REDUCE:
                            if operation.reduce(conf) != -1:
                                break
                        elif baseTransition == Transition.SHIFT:
                            if operation.shift(conf) != -1:
                                break
                    else:
                        raise ValueError(
                            "The predicted transition is not recognized, expected errors"
                        )

            # Finish with operations build the dependency graph from Conf.arcs

            new_depgraph = deepcopy(depgraph)
            for key in new_depgraph.nodes:
                node = new_depgraph.nodes[key]
                node["rel"] = ""
                # With the default, all the token depend on the Root
                node["head"] = 0
            for (head, rel, child) in conf.arcs:
                c_node = new_depgraph.nodes[child]
                c_node["head"] = head
                c_node["rel"] = rel
            result.append(new_depgraph)

        return result


In [13]:
!pip install pytextdist
from nltk.parse.dependencygraph import DependencyGraph
from nltk.parse import ProbabilisticProjectiveDependencyParser
from nltk.corpus import dependency_treebank
import nltk
from nltk.parse import DependencyEvaluator

nltk.download('dependency_treebank')

# print dependency graph in CoNLL format
#print(dependency_treebank.parsed_sents()[0].to_conll(10))

ppdp = ProbabilisticProjectiveDependencyParser()

# train parser on graphs
#ppdp.train(dependency_treebank.parsed_sents())

tp = TransitionParser('arc-standard')

modified_features = True

#SVM 
res = tp.train(dependency_treebank.parsed_sents()[:300], 'tp.model', modified_features=modified_features, spec_mod = 'svm')
#equivalent to the default value
#res = tp.train(dependency_treebank.parsed_sents()[:500], 'tp.model', modified_features=modified_features)
#NB: Any string passed to spec_mod which is not part of the ones listed before, will make the model execute the 'svm' model

#Decision Tree Classifier
#res = tp.train(dependency_treebank.parsed_sents()[:500], 'tp.model', modified_features=modified_features, spec_mod = 'DecisionTreeClassifier')

#Random Forest
#res =tp.train(dependency_treebank.parsed_sents()[:500], 'tp.model', modified_features=modified_features, spec_mod = 'RandomForestClassifier')

#Multi-Layer Perceptron
#res = tp.train(dependency_treebank.parsed_sents()[:500], 'tp.model', modified_features=modified_features, spec_mod = 'MLPClassifier')

# parsing takes a list of dependency graphs and a model as arguments
parses = tp.parse(dependency_treebank.parsed_sents()[-30:], 'tp.model', modified_features=modified_features)
de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-30:])
las, uas = de.eval()
print(las)
print(uas)


[nltk_data] Downloading package dependency_treebank to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package dependency_treebank is already up-to-date!
svm
[LibSVM]0.9024725274725275
0.9024725274725275
